In [ ]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    scalars_operation_algorithm,
)
from kale import theme

In [ ]:
# MESH_GEOMETRY_FILE_NAME = "nankai_very_very_dense.vtk"
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
# MESH_VALUES_FILE_NAME = "2023_03_03_16_15_24.hdf"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.01
TIME_STEP_INTERESTING = 55849
TIME_STEP_INTERESTING = 1000

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

In [ ]:
# Set which array to be active
engine.mesh.active_scalars_name = "geometric_moment"
engine.time_step = TIME_STEP_INTERESTING

In [ ]:
cpos = [
    (246.72654935831832, 56.640050977986135, 19.47264855846906),
    (235.64372937685064, 45.67443514953468, -0.9481135945038834),
    (-0.6591979033815945, -0.45237409343002455, 0.6006794517633125),
]

In [ ]:
engine.mesh

## Plot with time controls

In [ ]:
pl = pv.Plotter()
pl.add_mesh(
    engine.algorithm,
    cmap=theme.COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=engine.clim(),
    show_scalar_bar=True,
    scalar_bar_args=theme.SCALAR_BAR_OPTS,
)
pl.add_mesh(engine.boundary)
# pl.camera_position = cpos
show_ui(engine, pl)

In [ ]:
# Run cell to fetch current position
pl.camera_position

## Banded Contouring

In [ ]:
engine.mesh.active_scalars_name = "geometric_moment"

# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-2500, 2500, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl = pv.Plotter()
pl.add_mesh(engine.boundary)
actor = pl.add_mesh(
    contour,
    cmap=theme.COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
    n_colors=N_COLORS,
)
pl.add_scalar_bar(
    mapper=actor.mapper,
    # title=f"{MANIPULATED_ARRAY_NAME}",
    title=f"m",
    **theme.SCALAR_BAR_OPTS,
)
pl.add_mesh(edges)
pl.add_text(f"i = {1:05d}", name="time-step-label", font_size=theme.TIMESTEP_FONT_SIZE)

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")

# pl.camera_position = cpos
pl.camera.zoom(1.2)

# Drawing lines as a grid proxy
# mesh = pv.Line((0, 0, 0), (0, 0, 1))
# mesh.plot(color='k', line_width=10)

# Experimenting with lights
light = pv.Light((-2, 2, 0), (0, 0, 0), "white")
pl.add_light(light)
show_ui(engine, pl)

### 4-panel view (all variables)

This still needs some optimization to make look really nice but here is a start

In [ ]:
engine.keys

In [ ]:
pl = pv.Plotter(shape=(1, 4), window_size=(1024 * 2, 768))

for i, var in enumerate(engine.keys):
    pl.subplot(0, i)
    contour, edges = contour_banded(
        # TODO: set contour levels for each scalar, for now use N
        engine.algorithm,
        11,
        rng=engine.clim(var),
        scalars=var,
    )
    pl.add_mesh(
        contour,
        cmap=theme.COLOR_MAPS[var],
        clim=engine.clim(var),
        n_colors=11 - 1,
        show_scalar_bar=True,
        scalar_bar_args=theme.SCALAR_BAR_OPTS,
    )
    pl.add_mesh(edges)
    pl.add_mesh(engine.boundary)

pl.link_views()
pl.camera_position = cpos
show_ui(engine, pl)

## Save Videos

In [ ]:
# var = "cumulative_slip"
var = "geometric_moment"

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed)
contour, edges = contour_banded(
    engine.algorithm,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=var,
)
pl.add_mesh(
    contour,
    cmap=theme.COLOR_MAPS[var],
    clim=CLIM,
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=theme.SCALAR_BAR_OPTS,
)
pl.add_mesh(edges)
pl.add_mesh(engine.boundary)

pl.camera_position = cpos

pl.image_scale = 2
# Make a video over time steps
# save_movie(engine, plotter, f'{var}.mpg', framerate=24, quality=1)
step = 500
# step = 1
pl.open_movie(f"{var}.mp4", framerate=48, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"time step: {tindex:05d}", name="time-step-label", font_size=18)
    pl.write_frame()
pl.close()

# Updated video style

In [ ]:
var = "geometric_moment"

engine.mesh.active_scalars_name = var

# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-600, 600, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed)

contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl.add_mesh(
    contour,
    # cmap=COLOR_MAPS[var],
    # cmap="PuOr_r",
    cmap="RdYlBu_r",
    clim=CLIM,
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges)
pl.add_mesh(engine.boundary)

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")

pl.camera.zoom(1.2)
pl.image_scale = 2

# Make a video over time steps
step = 10
pl.open_movie(f"{var}_test.mp4", framerate=60, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"i = {tindex:05d}", name="time-step-label", position="lower_edge")
    pl.write_frame()
pl.close()